# Sistem Penjadwalan Praktikum Otomatis menggunakan Algoritma Genetika dan Tabu Search.

## Import Library

In [2]:
import os
import sys
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

## Import Model

In [3]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant
from collections import namedtuple

## Data extraction from model

In [4]:
class LaboratoryData:

    def __init__(self, lab = Laboratory):
        self._lab = lab
        self._laboratories = lab.objects.all()

    @property
    def laboratories(self): 
        return self._laboratories.values()
    
    def get_assistant(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        assistants = laboratory.assistants.all()
        return assistants
    
    def get_module(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        modules = laboratory.modules.all()
        return modules
    
class ModuleData:

        def __init__(self, module = Module):
            self._module = module
            self._modules = module.objects.all()
    
        @property
        def modules(self): 
            return self._modules.values()

class ParticipantData:

    def __init__(self, participant = Participant):
        self._participant = participant
        self._participants = participant.objects.all()
    
    @property
    def participants(self):
        return self._participants.values()
    
    def group(self, id):
        group_memberships = self._participants.filter(id=id).first().group_memberships.all()
        groups = []
        for group_membership in group_memberships:
            group = group_membership.group
            groups.append(group)
        return groups
    
class GroupData:
    def __init__(self, group = Group):
        self._group = group
        self._groups = group.objects.all()

    @property
    def groups(self):
        return self._groups.values()
    
    def get_group(self, id):
        group = self._groups.filter(id=id).first()
        return group
    
    def participants(self, id):
        group = self._groups.filter(id=id).first()
        group_memberships = group.group_memberships.all()
        participants = []
        for group_membership in group_memberships:
            participant = group_membership.participant
            participants.append(participant)
        return participants
    
    def get_participant_schedule(self, id):
        participants = self.participants(id)
        participant_schedule = []
        for participant in participants:
            participant_schedule.append(participant.regular_schedule)
        return participant_schedule
    
    def get_group_schedule(self, id):
        participant_schedule = self.get_participant_schedule(id)
        if len(participant_schedule) == 0:
            return {}
        days = participant_schedule[0].keys()
        merged_schedule = {day: {} for day in days}
        for day in days:
            for time_slot in participant_schedule[0][day]:
                is_available = all(schedule[day][time_slot] for schedule in participant_schedule)
                merged_schedule[day][time_slot] = is_available
        return merged_schedule
                
            
class AssistantData:

    def __init__(self, assistant = Assistant):
        self._assistant = assistant
        self._assistants = assistant.objects.all()

    @property
    def assistants(self):
        return self._assistants.values()

    
    def get_assistant(self, id):
        assistant = self._assistants.filter(id=id).first()
        return assistant
    
    def get_assistant_modules(self,id):
        assistant = self._assistant.filter(id=id).first()
        assistant_memberships = assistant.assistant_memberships.all()
        modules = []
        for assistant_membership in assistant_memberships:
            module = assistant_membership.module
            modules.append(module)
        return modules

## Genetic Algorithm

In [5]:
import pandas as pd

In [6]:
# Assigning data to variables
lab_data = LaboratoryData()
module_data = ModuleData()
participant_data = ParticipantData()
group_data = GroupData()
assistant_data = AssistantData()

In [7]:
pd.DataFrame(participant_data.participants)

,id,name,nim,semester_id,regular_schedule
0,5,Setio Ningrum,3332210016,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
1,6,Ruth Anandina,3332210051,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
2,7,ADHITAMA WIRA YUDHA,3332190029,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
3,8,FARHAN NUGROHO,3332190072,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
4,9,BAGAS NURYANTO,3332190080,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
...,...,...,...,...,...
102,55,Maulana Ali Akbar,3332210082,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."
103,65,Naga Tunggal,3332210004,1,"{'Friday': {'Shift1': True, 'Shift2': False, '..."
104,86,Randy eleanor,3332210057,1,"{'Friday': {'Shift1': True, 'Shift2': True, 'S..."
105,106,MAHESA NURUL VIKAR,3332200087,1,"{'Friday': {'Shift1': False, 'Shift2': True, '..."


In [8]:
pd.DataFrame(group_data.get_group_schedule(1))

,Friday,Monday,Tuesday,Saturday,Thursday,Wednesday
Shift1,False,True,True,False,True,False
Shift2,True,True,False,False,False,False
Shift3,False,False,False,True,False,False
Shift4,True,False,False,False,False,True
Shift5,False,True,False,True,True,True
Shift6,True,False,True,True,False,True


Gene Representation for Scheduling Problem

In [9]:
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

In [10]:
# Gene Representation
# 1. Laboratory
# 2. Module chapter
# 3. Group
# 4. Assistant
# 5. Time Slot : Date, Day, Time
# 6. Availability: json, e.g. {"Friday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Monday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Tuesday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": false}, "Saturday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": false, "Shift6": true}, "Thursday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": true, "Shift5": true, "Shift6": true}, "Wednesday": {"Shift1": true, "Shift2": true, "Shift3": true, "Shift4": false, "Shift5": false, "Shift6": true}}

class Gene:
    def __init__(self, lab, module_chapter, group, assistant, time_slot: TimeSlot):
        self.lab = lab
        self.module_chapter = module_chapter
        self.group = group
        self.assistant = assistant
        self.time_slot = time_slot
        
    def __repr__(self):
        return f"Laboratory: {self.lab}, Module Chapter: {self.module_chapter}, Group: {self.group}, Assistant: {self.assistant}, Time Slot: {self.time_slot}"
    
    def __eq__(self, other):
        return self.lab == other.lab and self.module_chapter == other.module_chapter and self.group == other.group and self.assistant == other.assistant and self.time_slot == other.time_slot
    
    def __hash__(self):
        return hash((self.lab, self.module_chapter, self.group, self.assistant, self.time_slot))
    
    def __str__(self) -> str:
        return f"Laboratory: {self.lab}, Module Chapter: {self.module_chapter}, Group: {self.group}, Assistant: {self.assistant}, Time Slot: {self.time_slot}"
    
    def generate(self):
        return Gene(self.lab, self.module_chapter, self.group, self.assistant, self.time_slot)
    

In [55]:
# Test Gene
gene = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], group_data.groups[0]["id"],assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift1"))
gene2 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], group_data.groups[1]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift2"))
gene3 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], group_data.groups[2]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Monday", "Shift1"))

In [12]:
gene.time_slot == gene3.time_slot

False

In [13]:
# Chromosome Representation
# 1. Gene
# 2. Fitness

class Chromosome:
    def __init__(self, genes, fitness):
        self.genes = genes
        self.fitness = fitness
        
    def __repr__(self):
        return f"Genes: {self.genes}, Fitness: {self.fitness}"
    
    def __eq__(self, other):
        return self.genes == other.genes and self.fitness == other.fitness
    
    def __hash__(self):
        return hash((self.genes, self.fitness))
    
    def __str__(self) -> str:
        return f"Genes: {self.genes}, Fitness: {self.fitness}"
    
    def generate(self):
        return Chromosome(self.genes, self.fitness)

In [57]:
# test chromosome
chromosome = Chromosome([gene, gene2, gene3], 0.5)

In [15]:
chromosome.generate()

Genes: [Laboratory: 1, Module Chapter: 1, Group: 1, Assistant: 1, Time Slot: TimeSlot(date='2021-01-01', day='Friday', shift='Shift1'), Laboratory: 1, Module Chapter: 1, Group: 2, Assistant: 1, Time Slot: TimeSlot(date='2021-01-01', day='Friday', shift='Shift2'), Laboratory: 1, Module Chapter: 1, Group: 3, Assistant: 1, Time Slot: TimeSlot(date='2021-01-01', day='Sunday', shift='Shift1')], Fitness: 0.5

In [16]:
# Objective Class for Fitness Function
# 1. Minimize Conflicts: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.

class MinimizeConflicts:
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.conflicts = namedtuple("Conflicts", ["time_slot", "lab"])
        self.conflicts_time_slot = set()
        self.assigned_time_slot = set()
        self.conflict_count = 0


    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Conflicts Time Slot: {self.conflicts_time_slot}, Assigned Time Slot: {self.assigned_time_slot}, Conflict Count: {self.conflict_count}"
    
    def __eq__(self, other):
        return self.chromosome == other.chromosome and self.conflicts_time_slot == other.conflicts_time_slot and self.assigned_time_slot == other.assigned_time_slot and self.conflict_count == other.conflict_count
    
    def calculate_conflicts(self):
        for gene in self.chromosome.genes:
            if (gene.lab, gene.time_slot) in self.assigned_time_slot:
                self.conflicts_time_slot.add(self.conflicts(gene.time_slot, gene.lab))
                self.conflict_count += 1
            else:
                self.assigned_time_slot.add((gene.lab, gene.time_slot))
        return self.conflict_count

    def calculate_fitness(self):
        self.calculate_conflicts()
        return self.conflict_count


In [17]:
a = MinimizeConflicts(chromosome)
a.calculate_fitness()

0

In [29]:
# 2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.

from collections import defaultdict
from itertools import combinations

class MaximizeResourceUtilization:
    def __init__(self, chromosome:list, max_groups_per_assistant, max_shift_per_assistant):
        self.chromosome = chromosome
        self.max_groups_per_assistant = max_groups_per_assistant
        self.max_shift_per_assistant = max_shift_per_assistant
        self.assistants_assignments = defaultdict(list)
        self.assistants_shifts = defaultdict(list)
        self.overload_penalty = 0

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Max Groups Per Assistant: {self.max_groups_per_assistant}, Max Shift Per Assistant: {self.max_shift_per_assistant}, Assistants Assignments: {self.assistants_assignments}, Assistants Shifts: {self.assistants_shifts}, Overload Penalty: {self.overload_penalty}"
    
    def calculate_assistants_assignments(self):
        self.assistants_assignments.clear()
        for gene in self.chromosome.genes:
            self.assistants_assignments[gene.assistant].append(gene.group)
        return self.assistants_assignments
    
    def calculate_assistants_shifts(self):
        self.assistants_shifts.clear()
        for gene in self.chromosome.genes:
            self.assistants_shifts[gene.assistant].append(gene.time_slot)
        return self.assistants_shifts
    
    def calculate_overload_penalty(self):
        self.overload_penalty = 0
        self.calculate_assistants_assignments()
        self.calculate_assistants_shifts()
        for assistant , assignments in self.assistants_assignments.items():
            assignments_penalty = max(0, len(assignments) - self.max_groups_per_assistant)
            shift_penalty = (max(0, len(self.assistants_shifts[assistant]) - self.max_shift_per_assistant))
            self.overload_penalty += assignments_penalty + shift_penalty
        return self.overload_penalty
    
    def calculate_fitness(self):
        self.calculate_overload_penalty()
        return self.overload_penalty
    
b = MaximizeResourceUtilization(chromosome, 2, 2)
b.calculate_fitness()

        

2

In [48]:
group_data.get_group_schedule(gene.group)

{'Friday': {'Shift1': False,
  'Shift2': True,
  'Shift3': False,
  'Shift4': True,
  'Shift5': False,
  'Shift6': True},
 'Monday': {'Shift1': True,
  'Shift2': True,
  'Shift3': False,
  'Shift4': False,
  'Shift5': True,
  'Shift6': False},
 'Tuesday': {'Shift1': True,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Saturday': {'Shift1': False,
  'Shift2': False,
  'Shift3': True,
  'Shift4': False,
  'Shift5': True,
  'Shift6': True},
 'Thursday': {'Shift1': True,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': True,
  'Shift6': False},
 'Wednesday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': True,
  'Shift5': True,
  'Shift6': True}}

In [51]:
group_data.get_group_schedule(gene.group)["Friday"]["Shift1"]

False

In [58]:
# 3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

class ParticipantAvailability:
    def __init__(self, chromosome, group_data: GroupData):
        self.chromosome = chromosome
        self.group_data = group_data
        self.unsatified_participants = defaultdict(list)
        self.unsatified_participants_count = 0

    def calculate_unsatisfied_participants(self):
        self.unsatified_participants.clear()
        for gene in self.chromosome.genes:
            participant_availability = group_data.get_group_schedule(gene.group)
            if not participant_availability[gene.time_slot.day][gene.time_slot.shift]:
                self.unsatified_participants[gene.group].append(gene.time_slot)
                self.unsatified_participants_count += 1
        return self.unsatified_participants_count
    
    def calculate_fitness(self):
        self.calculate_unsatisfied_participants()
        return self.unsatified_participants_count
    
c = ParticipantAvailability(chromosome, group_data)
c.calculate_fitness()
            

2

In [59]:
c.unsatified_participants

defaultdict(list,
            {1: [TimeSlot(date='2021-01-01', day='Friday', shift='Shift1')],
             3: [TimeSlot(date='2021-01-01', day='Monday', shift='Shift1')]})